# Data Indexing and Selection Examples

2020-09-28 - Jeff Smith

In [ ]:
# Getting things ready
import pandas as pd
from datetime import datetime

def show(data, show_data = 0):
    print ("   Type: {:}".format(type(data)))
    print ("  Index: {:}".format(data.index))
    print ("  Shape: {:}".format(data.shape))
    print ("Columns: {:}".format(data.columns))
    for col in data.columns:
        print ("    {:} ({:})".format(col,data[col].dtype))
    if show_data == 1:
        print(data.head())
    elif show_data == 2:
        print(data.values)
        
print("Pandas version: {:}".format(pd.__version__))

filepath = "../data/"

# US Economic Data dataset
## Default read_csv()

In [ ]:
# Start with the US Economic Data dataset.
df1 = pd.read_csv(filepath + '10_us_economic_data.csv')
show(df1,1)

In [ ]:
# Extracting a single column (as a Series)
# Month - Either syntax works since the column name is alpha and is not a keyword.
df1.Month
#df1["Month"]

In [ ]:
# Extracting multiple Columns
df1[['Month', 'UnemploymentRate']]
# note the double brackets

In [ ]:
# What about the first 12 entries (the first year of data)?
df1.loc[0:11]
# why does the row for index 11 show up?  For normal Python slices, it would not (why not?)
# Try iloc rather than loc.

In [ ]:
# What month had the highest unemployment rate?
rate = df1.UnemploymentRate.max()
df1.loc[df1.UnemploymentRate == rate]

In [ ]:
# or all at once
df1.loc[df1.UnemploymentRate == df1.UnemploymentRate.max()]

In [ ]:
# What months had the unemployment rate greater than or equal to 9.5
df1[df1.UnemploymentRate >= 9.5]
# Here the index expression is a mask -- see the next cell

In [ ]:
# the mask
df1.UnemploymentRate >= 9.5

In [ ]:
# What about the Unemployment Rate and Jobs Added numbers for 
# the year 2010?
df1[['Month', 'UnemploymentRate','JobsAdded']].loc[24:35]

In [ ]:
# What aobut the JobsAdded for the 12 month period after the month 
# with the maximum unemployment rate
# From the previous query, I knwo that max unemployment occured at loc 21
df1[['Month', 'JobsAdded', 'UnemploymentRate']].loc[22:33]

In [ ]:
# or -- by using the argmax ("argument for entry with maximum value") function:
df1[['Month', 'JobsAdded', 'UnemploymentRate']].loc[df1.UnemploymentRate.argmax()+1:df1.UnemploymentRate.argmax()+12]

## Enhanced call to read_csv()

In the default version of the DataFrame, the date was read as string objects.  This makes date arithmetic clunky.  Let's try reading them in as datetime objects and setting the month to be the DataFrame index.

In [ ]:
# Here, if I want to use the Month as the index AND specify that Pandas
# parse the dates for the index:
df2 = pd.read_csv(filepath + '10_us_economic_data.csv', parse_dates = ['Month'], index_col=0)
show(df2,1)

In [ ]:
# Note that we can alway using the native Python (implicit) referencing.
df2.iloc[0:12]

In [ ]:
# But now we can use dates.  Note also that we're using a string rather
# that a datetime object
df2.loc['2008-02-01']

In [ ]:
# what about a different date format?
df2.loc['02/01/08']

In [ ]:
# Now we can do a direct date range.
df2.loc['2012-01-01':'2012-12-01']

In [ ]:
df2.JobsAdded.loc['2012-01-01':'2012-12-01']